<a href="https://colab.research.google.com/github/khalilvandian/Fortune_company_opinion_mining/blob/main/Behavioral_Psychology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install emoji
!pip install langdetect
!pip install nltk
!pip install tqdm
!pip install langid

In [ ]:
import kagglehub
import os
import pandas as pd
import spacy
from google.colab import drive
import emoji
import re
from langdetect import detect, detect_langs
import nltk
import string
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from tqdm import tqdm
import langid
from ast import pattern
tqdm.pandas()

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Download latest version
# path = kagglehub.dataset_download("jarredgaudineer/social-media-posts-fortune-1000-companies")

# print("Path to dataset files:", path)

In [ ]:
# Using the 'path' variable from your previous code
# downloaded_files = os.listdir(path)

# print("Downloaded files:", downloaded_files)

In [ ]:
# temp_df = pd.read_json(os.path.join(path, downloaded_files[0]))

In [ ]:
# temp_df['tweets'][0]

In [ ]:
# combine path with the file name
# files_directory = path
# path = os.path.join(files_directory, "10Mar2025.json")

In [ ]:
# The List of Fortune 1000 Companies
# Download latest version
fort_list_path = kagglehub.dataset_download("jeannicolasduval/2024-fortune-1000-companies")

print("Path to dataset files:", fort_list_path)

for file in os.listdir(fort_list_path):
  print(file)

100%|██████████| 301k/301k [00:00<00:00, 635kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jeannicolasduval/2024-fortune-1000-companies/versions/1
fortune1000_2024.csv
fortune1000_2024.parquet
fortune1000_companyinfo.parquet
fortune1000_companies.parquet


In [ ]:
fortune_company_list = pd.read_csv(os.path.join(fort_list_path, "fortune1000_2024.csv"))
fortune_company_list.head()

,Rank,Company,Ticker,Sector,Industry,Profitable,Founder_is_CEO,FemaleCEO,Growth_in_Jobs,Change_in_Rank,...,Assets_M,CEO,Country,HeadquartersCity,HeadquartersState,Website,CompanyType,Footnote,MarketCap_Updated_M,Updated
0,1,Walmart,WMT,Retailing,General Merchandisers,yes,no,no,no,0.0,...,252399.0,C. Douglas McMillon,U.S.,Bentonville,Arkansas,https://www.stock.walmart.com,Public,"Figures are for fiscal year ended Jan. 31, 202...",559911.0,2024-08-05
1,2,Amazon,AMZN,Retailing,Internet Services and Retailing,yes,no,no,no,0.0,...,527854.0,Andrew R. Jassy,U.S.,Seattle,Washington,https://www.amazon.com,Public,"Market value as of July 15, 2024.",2005565.0,2024-08-05
2,3,Apple,AAPL,Technology,"Computers, Office Equipment",yes,no,no,no,1.0,...,352583.0,Timothy D. Cook,U.S.,Cupertino,California,https://www.apple.com,Public,"Figures are for fiscal year ended Sept. 30, 20...",3594309.0,2024-08-05
3,4,UnitedHealth Group,UNH,Health Care,Health Care: Insurance and Managed Care,yes,no,no,yes,1.0,...,273720.0,Andrew P. Witty,U.S.,Minnetonka,Minnesota,https://www.unitedhealthgroup.com,Public,"Market value as of July 15, 2024.",474339.0,2024-08-05
4,5,Berkshire Hathaway,BRKA,Financials,Insurance: Property and Casualty (Stock),yes,no,no,yes,2.0,...,1069978.0,Warren E. Buffett,U.S.,Omaha,Nebraska,https://www.berkshirehathaway.com,Public,"Market value as of July 15, 2024.",937028.0,2024-08-05


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Specify the file path within your Drive
# path = '/content/drive/MyDrive/companyDataSample.json'
path = '/content/drive/MyDrive/allCompanyData.json'

# Save the data as a JSON file
# with open(path, 'w') as f:
#     json.dump(data, f)

# print(f"Data saved to: {path}")

In [ ]:
import json

with open(path, 'r') as f:
    data = json.load(f)

In [ ]:
data[0].keys()

dict_keys(['company', 'rank', 'timestamp', 'tweets', 'posts', 'comments', 'metadata'])

In [ ]:
columns = ["Company", "Text", "Source"]
atts_temp = ["tweets", "comments", "posts"]
source_dic = {"tweets": "Twitter", "comments": "Reddit-Comments", "posts": "Reddit-Posts"}
mp_doc_list = []

for company in data:
  for text_list in atts_temp:
    if text_list in company:
      for text in company[text_list]:
        mp_doc_list.append([company["company"], text["text"], source_dic[text_list]])


data_df = pd.DataFrame(mp_doc_list, columns=columns)
data_df

,Company,Text,Source
0,Walmart,$GOOGL $GOOG $WMT Google And Walmart Payment A...,Twitter
1,Walmart,Can $PHNIX become the first meme coin to dethr...,Twitter
2,Walmart,"Walmart $WMT \n\nThe Laughing Cow Cheese, 32 p...",Twitter
3,Walmart,Some of the most iconic businesses ever \n\n1....,Twitter
4,Walmart,Walmart $WMT currently has 1.05 Billion Square...,Twitter
...,...,...,...
591342,Clear Channel Outdoor Hldgs.,Hi BUB Sub!\n\n\nHad this account on the back ...,Reddit-Posts
591343,Clear Channel Outdoor Hldgs.,I am curious how these events typically play o...,Reddit-Posts
591344,Clear Channel Outdoor Hldgs.,,Reddit-Posts
591345,Clear Channel Outdoor Hldgs.,The Role of the National Weather Service in Em...,Reddit-Posts


In [ ]:
enriched_data = pd.merge(fortune_company_list[["Company", "Ticker", "Sector", "Industry"]], data_df, left_on="Company", right_on="Company")
enriched_data.sample(10)

,Company,Ticker,Sector,Industry,Text,Source
183159,Visa,V,Business Services,Financial Data Services,,Reddit-Posts
19,Walmart,WMT,Retailing,General Merchandisers,Edit: asking because my walmart has about 22 o...,Reddit-Comments
59068,Target,TGT,Retailing,General Merchandisers,50 years? I’ve got a list of eternal bans. Com...,Reddit-Comments
335004,ServiceNow,NOW,Technology,Computer Software,Hello everyone my self rudra from Delhi ...a l...,Reddit-Posts
388816,PENN Entertainment,PENN,"Hotels, Restaurants & Leisure","Hotels, Casinos, Resorts",,Reddit-Posts
482485,Visteon,VC,Motor Vehicles & Parts,Motor Vehicles & Parts,"Basically, the tittle.\n\nI used to be diamond...",Reddit-Posts
587237,MSCI,MSCI,Business Services,Financial Data Services,"In diesem Jahr, vermutlich gegen Frühling würd...",Reddit-Posts
460362,Lyft,LYFT,Technology,Internet Services and Retailing,Currently in one on the way to WS and was told...,Reddit-Comments
289574,VF,VFC,Apparel,Apparel,I'd rather keep the Super Parts instead. The a...,Reddit-Comments
492148,Toast,TOST,Business Services,Financial Data Services,Hi Toast!\n\n![gif](giphy|fX5cZemSfX1cMZYuUJ),Reddit-Comments


In [ ]:
# compare unique companies in the enriched dataset and main dataset
enriched_companies = enriched_data["Company"].unique()
main_companies = data_df["Company"].unique()

print(f"Number of companies in the enriched dataset: {len(enriched_companies)}")
print(f"Number of companies in the main dataset: {len(main_companies)}")

Number of companies in the enriched dataset: 953
Number of companies in the main dataset: 953


In [ ]:
data_df = enriched_data

## Preprocessing the texts

In [ ]:
data_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,591347,579881,591347,591347,591347,591347
unique,953,932,21,75,487445,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,,Reddit-Posts
freq,9438,9438,132908,60577,82315,329493


In [ ]:
# show duplicates
data_df[data_df.duplicated()].sample(10)

,Company,Ticker,Sector,Industry,Text,Source
175475,Netflix,NFLX,Media,Entertainment,,Reddit-Posts
313821,Ebay,EBAY,Technology,Internet Services and Retailing,,Reddit-Posts
532870,Designer Brands,DBI,Retailing,Specialty Retailers: Apparel,I've been reading articles that all really hig...,Reddit-Posts
3996,Walmart,WMT,Retailing,General Merchandisers,,Reddit-Posts
562490,Roblox,RBLX,Media,Entertainment,,Reddit-Posts
38453,Ford Motor,F,Motor Vehicles & Parts,Motor Vehicles & Parts,,Reddit-Posts
54170,Comcast,CMCSA,Telecommunications,Telecommunications,\nThank you for joining us here on r/Comcast_X...,Reddit-Comments
443652,Snap,SNAP,Technology,Internet Services and Retailing,Tired of fa.ke accounts? Get free nudes instan...,Reddit-Comments
448210,Snap,SNAP,Technology,Internet Services and Retailing,,Reddit-Posts
287968,Ryder System,R,Transportation,Transportation and Logistics,Nosferatu movie review: The cinematography is ...,Reddit-Comments


In [ ]:
# drop duplicates
data_df = data_df.drop_duplicates()
data_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,490487,480398,490487,490487,490487,490487
unique,953,932,21,75,487445,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,,Reddit-Comments
freq,7372,7372,110765,49625,883,242717


### Sample for Analysis

In [ ]:
sample_data_df = data_df.copy()
sample_data_df.head()

,Company,Ticker,Sector,Industry,Text,Source
0,Walmart,WMT,Retailing,General Merchandisers,$GOOGL $GOOG $WMT Google And Walmart Payment A...,Twitter
1,Walmart,WMT,Retailing,General Merchandisers,Can $PHNIX become the first meme coin to dethr...,Twitter
2,Walmart,WMT,Retailing,General Merchandisers,"Walmart $WMT \n\nThe Laughing Cow Cheese, 32 p...",Twitter
3,Walmart,WMT,Retailing,General Merchandisers,Some of the most iconic businesses ever \n\n1....,Twitter
4,Walmart,WMT,Retailing,General Merchandisers,Walmart $WMT currently has 1.05 Billion Square...,Twitter


In [ ]:
def preprocess_text(text, index, targets):

  # First Demojize Text
  edited_text = emoji.replace_emoji(text, '')

  # Mask company
  try:
    pattern = r'(?<![a-zA-Z])(?:' + '|'.join(re.escape(word) for word in targets if isinstance(word, str)) + r')(?![a-zA-Z])'
    if re.search(pattern, text, flags=re.IGNORECASE):
        edited_text = re.sub(pattern, "TargetedCompany", text, flags=re.IGNORECASE)
    else:
        edited_text = ""

  except Exception as e:
    raise

  # lower case
  edited_text = edited_text.lower()

  # remove links, mentions, hashtags
  edited_text = re.sub(r"http\S+|www\S+|@\w+|#\w+", '', edited_text)

  # Remove numbers
  edited_text = re.sub(r'\d+', '', edited_text)

  # Remove non ASCII
  edited_text = re.sub(r'[^\x00-\x7F]+', '', edited_text)

  # remove extra spaces
  edited_text = re.sub(r'\s+', ' ', edited_text).strip()

  # return if text is empty
  if edited_text == "":
    return edited_text

  # Remove non english
  try:
    # lang = detect(edited_text)
    lang, _ = langid.classify(edited_text)
    if lang != "en":
        edited_text = ""
  except Exception as e:
    # print(f"Error detecting language for text at index {index} and content:{edited_text}: {e}")
    edited_text = ""

  return edited_text


In [ ]:
temp_df = sample_data_df.copy()
temp_df["Text"] = temp_df.progress_apply(lambda row: preprocess_text(row["Text"], row.name, [row["Company"], row["Ticker"]]), axis=1)
temp_df

100%|██████████| 490487/490487 [34:49<00:00, 234.72it/s]


,Company,Ticker,Sector,Industry,Text,Source
0,Walmart,WMT,Retailing,General Merchandisers,$googl $goog $targetedcompany google and targe...,Twitter
1,Walmart,WMT,Retailing,General Merchandisers,can $phnix become the first meme coin to dethr...,Twitter
2,Walmart,WMT,Retailing,General Merchandisers,targetedcompany $targetedcompany the laughing ...,Twitter
3,Walmart,WMT,Retailing,General Merchandisers,some of the most iconic businesses ever . $mcd...,Twitter
4,Walmart,WMT,Retailing,General Merchandisers,targetedcompany $targetedcompany currently has...,Twitter
...,...,...,...,...,...,...
591342,Clear Channel Outdoor Hldgs.,CCO,Business Services,"Advertising, marketing",,Reddit-Posts
591343,Clear Channel Outdoor Hldgs.,CCO,Business Services,"Advertising, marketing",,Reddit-Posts
591344,Clear Channel Outdoor Hldgs.,CCO,Business Services,"Advertising, marketing",,Reddit-Posts
591345,Clear Channel Outdoor Hldgs.,CCO,Business Services,"Advertising, marketing",,Reddit-Posts


### Remove All blank

In [ ]:
temp_df = temp_df[temp_df["Text"] != ""]
temp_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,345786,336538,345786,345786,345786,345786
unique,919,898,21,75,339469,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,[,Reddit-Comments
freq,6841,6841,87823,40699,278,174806


### Wrap up pre-processing

In [ ]:
temp_df.describe()

,Company,Ticker,Sector,Industry,Text,Source
count,345786,336538,345786,345786,345786,345786
unique,919,898,21,75,339469,3
top,Tesla,TSLA,Technology,Internet Services and Retailing,[,Reddit-Comments
freq,6841,6841,87823,40699,278,174806


Saving the preprocesed data into the drive.

In [ ]:
path = '/content/drive/MyDrive/preprocessedCompanyData.json'

# # Save the data as a JSON file
# with open(path, 'w') as f:
#     json.dump(data, f)

# read the saved preprocessed data
with open(path, 'r') as f:
    preprocessed_text_data = json.load(f)


## Vectorization and Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
target_filter = "targetedcompany"

def tokenize_text(text):

  # first remove punct
  no_punct = [char for char in text if char not in string.punctuation]
  no_punct = "".join(no_punct)

  # replace ’ with '
  no_punct = no_punct.replace("’", "'")

  # Lemmatize
  doc = no_punct.split()

  tokens = []
  for token in doc:
    lemmed = lemmatizer.lemmatize(token)
    if lemmed not in stop_words and lemmed != target_filter:
      tokens.append(lemmed)

  return tokens

In [ ]:
tokenized_texts = temp_df.sample(10, random_state=28).apply(lambda row: tokenize_text(row["Text"]), axis=1)

In [ ]:
tokenized_texts

,0
446134,"[possible, disable, building, time, useful, so..."
335304,"[youre, fence, ordering, tesla, targetedcompan..."
209031,"[hi, name, disclosure, ive, researching, issue..."
81658,"[mufasa, lion, king, picture, week, weekend, r..."
470657,"[five, week, oh, ross, procedure, thing, mostl..."
404454,"[pen, la, time, crossword, clue, guide, la, ti..."
213944,"[brother, ill, happy, support, match, except, ..."
396252,"[want, use, port, pair, gpu, using, link, fsg,..."
342478,"[drove, gamespot, opened, like, wa, online, dr..."
144735,"[join, discord, pay, sub, get, monitor, notifi..."


In [ ]:
vectorizer = CountVectorizer(analyzer=tokenize_text, min_df=0.005, max_df=0.995)
bow = vectorizer.fit_transform(temp_df["Text"])

KeyboardInterrupt: 

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(f"Shape of the BOW: {bow.shape}")
print(f"Number of non-zero values: {bow.nnz}")

In [ ]:
tf_idf = TfidfTransformer()

In [ ]:
tf_idf_text = tf_idf.fit_transform(bow)

In [ ]:
tf_idf_text.shape

## Topic Modeling The texts
Initially, we need to understand what sort of content we have. This will help deciding if we should go with data based topics or anchored topics.

In [ ]:
lda = LatentDirichletAllocation(n_components=6, max_iter=10, learning_method='batch', random_state=42, verbose=1)

In [ ]:
lda_op = lda.fit_transform(tf_idf_text)

In [ ]:
lda_op.shape

In [ ]:
lda_op[0]

In [ ]:
H1 = lda.components_

In [ ]:
num_words = 20
vocab = np.array(vectorizer.get_feature_names_out())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [ ]:
topics

In [ ]:
lda_op

In [ ]:
temp_df["Topic"] = np.argmax(lda_op, axis=1)

In [ ]:
temp_df

In [ ]:
path = '/content/drive/MyDrive/allCompanyData_TopicModeled.csv'
# temp_df.to_csv(path)
# print(f"Data saved to: {path}")

In [ ]:
# Map topic index to a descriptive title
topic_titles = {
    0: "Casual Online Chatter",
    1: "Book or Media Reviews",
    2: "Consumer Goods / Retail",
    3: "Finance & Business News",
    4: "Job Applications & Recruitment",
    5: "Tech Support & Software Use"
}


In [ ]:
import matplotlib.pyplot as plt

# Count topics
topic_counts = temp_df["Topic"].value_counts().sort_index()

# Map the index to topic titles for labeling
labels = [topic_titles[i] for i in topic_counts.index]

# Plot with titles as x-axis labels
plt.figure(figsize=(10, 6))
plt.bar(labels, topic_counts.values)
plt.xticks(rotation=45, ha='right')
plt.title("Document Counts per Topic")
plt.ylabel("Number of Documents")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Group data by sector and topic, and count occurrences
sector_topic_counts = temp_df.groupby(['Sector', 'Topic'])['Topic'].count().unstack()

# Get sector and topic labels
sectors = sector_topic_counts.index
topic_labels = [topic_titles[i] for i in range(lda.n_components)]

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot bars for each topic within each sector
bottom = [0] * len(sectors)  # Initialize bottom for stacking
for i, topic_label in enumerate(topic_labels):
    ax.bar(sectors, sector_topic_counts[i], label=topic_label, bottom=bottom)
    bottom = [bottom[j] + sector_topic_counts[i][j] for j in range(len(sectors))]

# Set chart labels and formatting
ax.set_xlabel("Sector")
ax.set_ylabel("Number of Documents")
ax.set_title("Topic Distribution Across Sectors")
ax.legend(title="Topics", bbox_to_anchor=(1.05, 1), loc='upper left')  # Place legend outside the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your DataFrame is named 'temp_df' and contains 'Source' and 'Topic' columns

# Group data by source and topic, and count occurrences
source_topic_counts = temp_df.groupby(['Source', 'Topic'])['Topic'].count().unstack()

# Get source and topic labels
sources = source_topic_counts.index
topic_labels = [topic_titles[i] for i in range(lda.n_components)]  # Assuming you have topic_titles defined

# Create the stacked bar chart
fig, ax = plt.subplots(figsize=(12, 6))

# Plot bars for each topic within each source
bottom = [0] * len(sources)  # Initialize bottom for stacking
for i, topic_label in enumerate(topic_labels):
    ax.bar(sources, source_topic_counts[i], label=topic_label, bottom=bottom)
    bottom = [bottom[j] + source_topic_counts[i][j] for j in range(len(sources))]

# Set chart labels and formatting
ax.set_xlabel("Source")
ax.set_ylabel("Number of Documents")
ax.set_title("Topic Distribution Across Sources")
ax.legend(title="Topics", bbox_to_anchor=(1.05, 1), loc='upper left')  # Place legend outside the plot
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Get feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Create a dictionary to store the word clouds for each topic
topic_wordclouds = {}

# Iterate through each topic
for topic_index in range(lda.n_components):
    # Get documents belonging to the current topic
    topic_documents = temp_df[temp_df["Topic"] == topic_index]["Text"]

    # Create a frequency dictionary for words in the topic documents
    word_frequencies = {}
    for doc in topic_documents:
        for word in tokenize_text(doc):  # Use your tokenize_text function
            if word in feature_names:
                word_frequencies[word] = word_frequencies.get(word, 0) + 1

    # Create a WordCloud object
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_frequencies)

    # Store the word cloud in the dictionary
    topic_wordclouds[topic_index] = wordcloud

    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for Topic {topic_index}: {topic_titles[topic_index]}")  # Assuming you have topic_titles
    plt.show()


## Test extract ADJ

Text is there now, we need to extract the dimensions for each of the companies.

In [ ]:
dimensions = ["Effectiveness", "Innovation", "Ethics", "Empathy", "Aesthetic"]

In [ ]:
import spacy
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict
import torch

# Load models
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer("all-MiniLM-L6-v2")

# Define your semantic dimensions with positive/negative poles
dimensions = {
    "effectiveness": ("efficient", "inefficient"),
    "innovation": ("innovative", "outdated"),
    "trust": ("honest", "manipulative"),
    "empathy": ("caring", "cold"),
    "aesthetic": ("sleek", "ugly")
}

# Pre-encode the pole vectors for speed
anchor_vectors = {
    dim: {
        "pos": model.encode(pos, convert_to_tensor=True),
        "neg": model.encode(neg, convert_to_tensor=True)
    }
    for dim, (pos, neg) in dimensions.items()
}

In [ ]:
def score_adjective_on_dimensions(adj):
    vec = model.encode(adj, convert_to_tensor=True)
    scores = {}
    for dim, anchors in anchor_vectors.items():
        pos_sim = util.cos_sim(vec, anchors["pos"])
        neg_sim = util.cos_sim(vec, anchors["neg"])
        score = (pos_sim - neg_sim).item()
        scores[dim] = score
    return scores


In [ ]:
def extract_adjectives(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.pos_ == "ADJ"]


In [ ]:
company_profiles = defaultdict(lambda: defaultdict(list))

# for company, posts in data.items():
#     for post in posts:
#         adjectives = extract_adjectives(post)
#         for adj in adjectives:
#             dim_scores = score_adjective_on_dimensions(adj)
#             for dim, score in dim_scores.items():
#                 company_profiles[company][dim].append(score)

for company in preprocessed_text_data:
  for text in preprocessed_text_data[company]:
    adjectives = extract_adjectives(text)
    for adj in adjectives:
      dim_scores = score_adjective_on_dimensions(adj)
      for dim, score in dim_scores.items():
        company_profiles[company][dim].append(score)


KeyboardInterrupt: 

In [ ]:
import numpy as np

final_scores = {}
for company, dims in company_profiles.items():
    final_scores[company] = {
        dim: round(np.mean(scores), 3)
        for dim, scores in dims.items() if scores
    }

from pprint import pprint
pprint(final_scores)


In [ ]:
# # 5 random items from list
# import random
# sample = random.sample(preprocessed_text_data["Walmart"], 10)

# for samp in sample:
#   print("============================ THE TEXT IS =============================")
#   print(samp)
#   print("                            THE ADJECTIVES ARE                        ")
#   print(extract_adjectives(samp))
#   print("==================================================================")

In [ ]:
preprocessed_text_data["Walmart"][2000]

## Sentence based

In [ ]:
company_profiles_wholeText = defaultdict(lambda: defaultdict(list))

for company in preprocessed_text_data:
  for text in preprocessed_text_data[company]:
    dim_scores = score_adjective_on_dimensions(text)
    for dim, score in dim_scores.items():
      company_profiles_wholeText[company][dim].append(score)

In [ ]:
# dimensions of a dictionary
company_profiles_wholeText.keys()

In [ ]:

final_scores_wholeText = {}
for company, dims in company_profiles_wholeText.items():
    final_scores_wholeText[company] = {
        dim: round(np.mean(scores), 3)
        for dim, scores in dims.items() if scores
    }

from pprint import pprint
pprint(final_scores_wholeText)
